In [87]:
import pandas as pd
import glob
from datetime import datetime

In [88]:
c_path = "C:\\Users\\ASUS\\OneDrive\\Máy tính\\KHTN_HK2\\ETL_Final_DE4\\"

In [89]:
def extract_file_json(load_data):
    data_frame = pd.read_json(load_data, lines = False)
    return data_frame

In [90]:
def extract():
    extracted_data_list = []  # Danh sách các DataFrame tạm thời
    json_path = c_path + "datasource"
    for json_file in glob.glob(json_path + "/*.json"):
        extracted_data = extract_file_json(json_file)
        extracted_data_list.append(extracted_data)
    extract_data = pd.concat(extracted_data_list, ignore_index=True)  # Kết hợp các DataFrame tạm thời
    return extract_data

In [91]:
path_file_csv = c_path + "datasource\\exchange_rates.csv"
file_csv = pd.read_csv(path_file_csv)


In [92]:
df = pd.DataFrame(file_csv)
df.head()

,Unnamed: 0,Rates
0,AUD,1.297088
1,BGN,1.608653
2,BRL,5.409196
3,CAD,1.271426
4,CHF,0.886083


In [93]:
df.rename(columns={'Unnamed: 0':'index_column'}, inplace = True)
df.head()

,index_column,Rates
0,AUD,1.297088
1,BGN,1.608653
2,BRL,5.409196
3,CAD,1.271426
4,CHF,0.886083


In [94]:
exchange_rate_GBP = df.loc[df['index_column'] == 'GBP', 'Rates'].values[0]
print(exchange_rate_GBP)

0.7323984208000001


In [95]:
'''
Changes the Market Cap (US$ Billion) column from USD to GBP
Rounds the Market Cap (US$ Billion)` column to 3 decimal places
Rename Market Cap (US$ Billion) to Market Cap (GBP$ Billion)
'''
def transform(data):
    data['Market Cap (US$ Billion)'] = data['Market Cap (US$ Billion)']*exchange_rate_GBP
    data['Market Cap (US$ Billion)'] = round(data['Market Cap (US$ Billion)'], 3)
    data.rename(columns = {'Market Cap (US$ Billion)':'Market Cap (GPD$ Billion)'}, inplace = True)
    return data

In [96]:
extracted_data = extract()
extracted_data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


In [97]:
trans = transform(extracted_data)
trans.head()

,Name,Market Cap (GPD$ Billion)
0,JPMorgan Chase,286.319
1,Industrial and Commercial Bank of China,252.834
2,Bank of America,238.272
3,Wells Fargo,225.588
4,China Construction Bank,188.519


In [98]:
def load(load_data):
    csv_path = c_path + "bank_market_cap_gbp.csv"
    load_data.to_csv(csv_path, index = False)
    return None

In [99]:
def log(message):
    time_format = '%H:%M:%S-%h-%d-%Y'
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(time_format)
    with open("API_logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

In [101]:
log("Extract json file")
extracted_data = extract()
log("Extracted json file")

log("Transform data")
trans = transform(extracted_data)
log("Transformed data")

log("Load data")
load(trans)
log("Loaded data")